# Faces in the Wild with GCN Semantic Segmentation
This code snippet runs the gcn semantic segmentation network over the webcam.

Play with:

1. A hat, a cap, or any wearable on top of your head.
2. Take some sunglasses on.
3. Try with moustache or beard.
4. Put a mouth-mask on (or simulate it with a napkin).

*Eurecat 2019 - Rafael Redondo*

In [118]:
# Make sure your network has been trained with this architectural parameters
target_size = 256
num_classes = 7
num_levels = 3

# Download from https://eurecatcloud.sharepoint.com/:u:/r/sites/audiovisualteam/Shared%20Documents/ELOF/FacesInTheWild/Checkpoints/gcn-dataset_3-levels_3-lr_0.0001-lrdecay_0.1-lrpatience_10-wdecay_0.0005-momentum_0.99-dataaugmen/gcn-epoch_0480.pth?csf=1&e=vjEPyY
checkpoint = "./data/gcn-epoch_0480.pth"

# Colorize your labeled classes
label_colors = [
    (0, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 0),
    (255, 0, 0),
    (255, 0, 255),
    (0, 255, 255)]

# Model loading (Resnet may take a while to download)
import torch
from torch.autograd import Variable
from torchvision.transforms import ToTensor, Normalize
import torch.nn.functional as F
import sys
sys.path.insert(1,'../pytorch-segmentation')
from gcnX import FCN

model = torch.nn.DataParallel(FCN(num_classes, num_levels))
model.load_state_dict(torch.load(checkpoint))
model.cuda()
model.eval()
;

from PIL import Image
import numpy as np
import cv2
from io import BytesIO as StringIO
from IPython import display

def showFrame(a, fmt='jpeg'):
    f = StringIO()
    Image.fromarray(a).save(f, fmt)
    display.display(display.Image(data=f.getvalue()))
    
vc = cv2.VideoCapture(0)

if not vc.isOpened():
 
    print("Error opening webcam")
    
else:
    try:
        while True:

                capturing, image = vc.read()
                
                if not capturing:
                    break
                    
                offset = int((image.shape[1] - image.shape[0]) * 0.5)
                image = image[:, offset:offset + image.shape[0]]
                image = np.flip(image,1)
                image = Image.fromarray(np.uint8(image))
                image = image.resize((target_size, target_size), Image.BILINEAR)

                # Pass forward
                img = ToTensor()(image)
                img = Normalize([.485, .456, .406], [.229, .224, .225])(img)
                img = Variable(img).cuda().unsqueeze(0)
                scores = model(img)  # first image in batch
                label_probs = F.log_softmax(scores[0], dim=0).cpu().detach().numpy()

                # Composite
                segments = np.zeros((target_size, target_size, 3))
                labels = np.argmax(label_probs, axis=0)
                composite = np.array(image)
                for l in range(len(label_probs)):
                    indexes = labels == l
                    for c in range(3):
                        segments[:, :, c][indexes] = label_colors[l][c]               
                        #composite[:,:,c][indexes] = (1-label_probs[l][indexes]) * composite[:,:,c][indexes] + \
                         #                           (a * composite[:,:,c][indexes] + (1-a) * label_colors[l][c]) * label_probs[l][indexes]
                

                a = 0.6     # the smaller the more intense the blending is (more greenish)
                image = np.array(image)
                composite = segments * (1-a) + image * a
                for c in range(3):
                    indexes = labels == 0
                    composite[:,:,c][indexes] = image[:,:,c][indexes]
                    
                composite = np.flip(np.array(composite),2)
                showFrame(composite.astype('uint8'))
                display.clear_output(wait=True)
            
    except KeyboardInterrupt:
        vc.release()
        print "Stream stopped"

Stream stopped
